<a href="https://colab.research.google.com/github/hira-ayub/Amazon_Vine_Analysis/blob/main/Vine_Review_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import pandas as pd
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.2'
spark_version = 'spark-3.1.2'
os.environ['SPARK_VERSION']=spark_version
# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark
# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"
# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:3 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:4 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:5 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:6 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:7 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:8 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:9 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:11 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:13 https://developer.download.nvidia.com/compute/machine-le

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-Challenge").getOrCreate()

In [3]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Outdoors_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Outdoors_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show()
df.count()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   18446823|R35T75OLUGHL5C|B000NV6H94|     110804376|Stearns Youth Boa...|        Outdoors|          4|            0|          0|   N|                Y|          Four Stars|          GOOD VALUE| 2015-08-31|
|         US|   13724367|R2BV735O46BN33|B000IN0W3Y|     624096774|Primal Wear Men's...|        Outdoors|          5|    

2302401

In [4]:
# Clean Data into vine_table DataFrame
vine_df = df.select(["review_id", "product_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
vine_df.show()
vine_df.count()

+--------------+----------+-----------+-------------+-----------+----+-----------------+
|     review_id|product_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+----------+-----------+-------------+-----------+----+-----------------+
|R35T75OLUGHL5C|B000NV6H94|          4|            0|          0|   N|                Y|
|R2BV735O46BN33|B000IN0W3Y|          5|            0|          0|   N|                Y|
|R2NBEUGPQQGXP1|B008RBJXFM|          4|            0|          0|   N|                Y|
|R17LLAOJ8ITK0S|B00FK8WUQY|          3|            1|          1|   N|                Y|
|R39PEQBT5ISEF4|B00EZA3VW0|          1|            0|          0|   N|                Y|
|R3GNM3SU9VHJFT|B006JA8WEG|          4|            1|          1|   N|                Y|
| R2Y81OP0EK467|B002PWFSEO|          5|            0|          0|   N|                Y|
|R2LFGSI6HAYH5F|B002DZGKHW|          5|            1|          1|   N|                Y|
|R297G6ED1IQO7W|B00AB

2302401

In [5]:
#Create dataframe of items with total votes over 20
voteover_20 = vine_df.filter("total_votes>=20")
voteover_20.show()
voteover_20.count()

+--------------+----------+-----------+-------------+-----------+----+-----------------+
|     review_id|product_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+----------+-----------+-------------+-----------+----+-----------------+
|R2FP3U4NHNFNL2|B00YPISPNC|          5|           25|         29|   N|                Y|
|R1UUK1977O38MU|B00T8NEI3A|          5|           31|         31|   N|                Y|
| RXO216LWUDV6O|B00FLTZ2ZS|          3|           29|         31|   N|                Y|
|R3NMJF7EBMM19V|B007SZ4XJ4|          3|           26|         27|   N|                Y|
|R2ZY0ZBDUO0XUY|B00NAINBM8|          3|           20|         21|   N|                Y|
| RBDVHX7PMQEKM|B00OAUPSSA|          5|           35|         39|   N|                Y|
| RGO3KSDXYWTGU|B00V65XX0C|          4|           26|         26|   N|                Y|
|R2O4P39D0ALCQK|B00NPLSZF8|          2|           14|         43|   N|                Y|
|R1E1PQGHWGO3Q1|B007Z

43574

In [6]:
# Create a helpful votes dataframe
helpful_vote = voteover_20.filter(voteover_20['helpful_votes']/voteover_20['total_votes']>=.5)
helpful_vote.show()
helpful_vote.count()

+--------------+----------+-----------+-------------+-----------+----+-----------------+
|     review_id|product_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+----------+-----------+-------------+-----------+----+-----------------+
|R2FP3U4NHNFNL2|B00YPISPNC|          5|           25|         29|   N|                Y|
|R1UUK1977O38MU|B00T8NEI3A|          5|           31|         31|   N|                Y|
| RXO216LWUDV6O|B00FLTZ2ZS|          3|           29|         31|   N|                Y|
|R3NMJF7EBMM19V|B007SZ4XJ4|          3|           26|         27|   N|                Y|
|R2ZY0ZBDUO0XUY|B00NAINBM8|          3|           20|         21|   N|                Y|
| RBDVHX7PMQEKM|B00OAUPSSA|          5|           35|         39|   N|                Y|
| RGO3KSDXYWTGU|B00V65XX0C|          4|           26|         26|   N|                Y|
|R1E1PQGHWGO3Q1|B007ZYIFQ0|          1|           89|         96|   N|                Y|
|R1XOXXUVYZEDJB|B0087

39976

In [7]:
# Create a dataframe of paid reviews
paid_reviews = helpful_vote.filter(helpful_vote["vine"] == "Y")
paid_reviews.show()

+--------------+----------+-----------+-------------+-----------+----+-----------------+
|     review_id|product_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+----------+-----------+-------------+-----------+----+-----------------+
|R3KPC0NBUDASX3|B00R8KC02Q|          5|           25|         25|   Y|                N|
|R119P2A95GGXX4|B00NOYKVSK|          5|           26|         28|   Y|                N|
|R1HKIRME8AJ89Z|B00GK4LUXQ|          5|           79|         82|   Y|                N|
|R3FY3GMBGOBR22|B00NFCFDRA|          5|           12|         20|   Y|                N|
|R15KH3FBSVYGBU|B00NFCFIR0|          5|           32|         37|   Y|                N|
|R22XZEQLVLWW49|B00NFCFAYQ|          5|           28|         33|   Y|                N|
|R2OIQYVAL9D63R|B00NFCFIR0|          4|           59|         65|   Y|                N|
|R1W3KCDHM6BMZT|B00G6KURDI|          4|           37|         38|   Y|                N|
|R3KDMPS96IJ10R|B00R8

In [8]:
# Create a dataframe of unpaid reviews
unpaid_reviews = helpful_vote.filter(helpful_vote["vine"] == "N")
unpaid_reviews.show()

+--------------+----------+-----------+-------------+-----------+----+-----------------+
|     review_id|product_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+----------+-----------+-------------+-----------+----+-----------------+
|R2FP3U4NHNFNL2|B00YPISPNC|          5|           25|         29|   N|                Y|
|R1UUK1977O38MU|B00T8NEI3A|          5|           31|         31|   N|                Y|
| RXO216LWUDV6O|B00FLTZ2ZS|          3|           29|         31|   N|                Y|
|R3NMJF7EBMM19V|B007SZ4XJ4|          3|           26|         27|   N|                Y|
|R2ZY0ZBDUO0XUY|B00NAINBM8|          3|           20|         21|   N|                Y|
| RBDVHX7PMQEKM|B00OAUPSSA|          5|           35|         39|   N|                Y|
| RGO3KSDXYWTGU|B00V65XX0C|          4|           26|         26|   N|                Y|
|R1E1PQGHWGO3Q1|B007ZYIFQ0|          1|           89|         96|   N|                Y|
|R1XOXXUVYZEDJB|B0087

In [9]:
#total number of Paid reviews
print("Paid Reviews:%s" % paid_reviews.count())

#number of 5-star reviews
print("5-star Paid Reviews:%s" % paid_reviews.filter("star_rating==5").count())

#% of 5 star reviews
print("Percent of 5-star Paid Reviews:%s" % "{:.2%}".format(paid_reviews.filter("star_rating==5").count()/paid_reviews.count()))

Paid Reviews:107
5-star Paid Reviews:56
Percent of 5-star Paid Reviews:52.34%


In [10]:

#total number of unpaid reviews
print("Unpaid Reviews:%s" % unpaid_reviews.count())

#number of 5-star reviews
print("5-star Unpaid Reviews:%s" % unpaid_reviews.filter("star_rating==5").count())

#% of 5 star reviews
print("Percent of 5-star Unpaid Reviews:%s" % "{:.2%}".format(unpaid_reviews.filter("star_rating==5").count()/unpaid_reviews.count()))

Unpaid Reviews:39869
5-star Unpaid Reviews:21005
Percent of 5-star Unpaid Reviews:52.69%
